In [1]:
from google.cloud import bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"service-acc/service_acc.json"
print("Service Account KEY:", os.environ['GOOGLE_APPLICATION_CREDENTIALS'])


Service Account KEY: service-acc/service_acc.json


In [2]:
#!pip install pandas
#!pip install db-dtypes

### Run query with config

In [3]:

client = bigquery.Client()
sql = """
    SELECT name
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    WHERE state = @state
    LIMIT @limit
"""

query_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter('state', 'STRING', 'TX'),
        bigquery.ScalarQueryParameter('limit', 'INTEGER', 100)
    ]
)



DefaultCredentialsError: File service-acc/service_acc.json was not found.

In [ ]:
# Run a Standard SQL query using the environment's default project
df = client.query(sql, job_config=query_config).to_dataframe()

In [ ]:
df

### Loading a pandas DataFrame to a BigQuery table

In [4]:
!pip install pyarrow

In [5]:
import pandas

df = pandas.DataFrame(
    {
        'my_string': ['a', 'b', 'c'],
        'my_int64': [1, 2, 3],
        'my_float64': [4.0, 5.0, 6.0],
        'my_timestamp': [
            pandas.Timestamp("1998-09-04T16:03:14"),
            pandas.Timestamp("2010-09-13T12:03:45"),
            pandas.Timestamp("2015-10-02T16:00:00")
        ],
    }
)

In [10]:
client = bigquery.Client()
table_id = 'sktblog.ths_test_big_dataset.ths_test_big_tb'
# Since string columns use the "object" dtype, pass in a (partial) schema
# to ensure the correct BigQuery data type.
job_config = bigquery.LoadJobConfig(schema=[
    bigquery.SchemaField("my_string", "STRING"),
])


In [11]:
job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)

In [12]:
# Wait for the load job to complete.
job.result()

Forbidden: 403 Billing has not been enabled for this project. Enable billing at https://console.cloud.google.com/billing. Table expiration time must be less than 60 days while in sandbox mode.